In [1]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import pdb
import requests
from time import sleep
from random import randint
import csv
import pandas as pd

In [2]:
# Defining Components for Review Scrape 
def build_rating(review):
    return str(review).split("<span class=\"a-icon-alt\">")[1].split("</span>")[0].split(" ")[0]

def build_title(review):
    return str(review).split("data-hook=\"review-title\"")[1].split("<span>")[1].split("</span>")[0]

def build_date(review):
    return str(review).split("data-hook=\"review-date\">")[1].split("</span>")[0].split("on ")[1]

def build_verified_purchase(review):
    try: 
        str(review).split('data-hook="avp-badge-linkless"')[1].split("</span>")[0]
        return 1
    except:
        return 0
    
def build_body(review):
    body = str(review).split("data-hook=\"review-body\">")[1].split("<span>")[1].split("</span>")[0] + "\n"
    # to remove <br>, <br/> and </br>
    body = body.replace("<br>", ".").replace("<br/>", ".").replace("</br>", ".").strip()
    return body

def review_format(values, reviews):
    for review in reviews:
                csv_body = []

                # Star Rating
                rating = build_rating(review)
                csv_body.append(int(float(rating)))

                # Title
                title = build_title(review)
                csv_body.append(title)

                # Date
                date = build_date(review)
                csv_body.append(date)

                # Verified Purchase
                verified_purchase = build_verified_purchase(review)
                csv_body.append(verified_purchase)

                # Body
                body = build_body(review)
                csv_body.append(body)

                values.append(csv_body)
    return values

In [3]:
def scrape(values, adjusted_url, given_url):
    """
    Scrapes and formats review features
    """
    headers = {'authority': 'www.amazon.com',
                   'pragma': 'no-cache',
                   'cache-control': 'no-cache',
                   'dnt': '1',
                   'upgrade-insecure-requests': '1',
                   'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
                   'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                   'sec-fetch-site': 'none',
                   'sec-fetch-mode': 'navigate',
                   'sec-fetch-dest': 'document',
                   'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8'
                  }

    # Download the page using requests
    #print("Downloading %s"%given_url)
    r = requests.get(adjusted_url, headers=headers)

    # Simple check to check if page was blocked (Usually 503)
    if r.status_code > 500:
        if "To discuss automated access to Amazon data please contact" in r.text:
            print("Page %s was blocked by Amazon. Please try using better proxies\n"%given_url)
        else:
            print("Page %s must have been blocked by Amazon as the status code was %d"%(given_url, r.status_code))

    soup = BeautifulSoup(r.text, 'html.parser')
    reviews = soup.find_all("div", attrs={"class": "a-section review aok-relative"})

    values = review_format(values, reviews)

    return values

In [4]:
def scraper_main(given_url, start_page, end_page, delay = 4):
    if end_page < start_page:
        print("Start page cannot be greater than end page. Please try again.")
        exit()
    
    values = []
    print(given_url)
    # Initialising starting position of scrape
    while start_page <= end_page:
        try:
            adjusted_url = given_url + "&pageNumber=" + str(start_page)
        except:
            print("Cannot read your URL. Please try again.")
            exit()
    
        # Delaying to prevent amazon block 
        delay = randint(0, int(delay))
        sleep(delay)

        values = scrape(values, adjusted_url, given_url)

        start_page += 1
    
    return values